In [2]:
# !pip install auto-gptq
# !pip install optimum
# !pip install bitsandbytes

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

Already Quantized model

In [3]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda:1",
    trust_remote_code=False,
    revision="main")

CUDA extension not installed.
CUDA extension not installed.
/DATA/sriparna/TA/Daivik/.conda/lib/python3.9/site-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'mode

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

To See output on your data

In [7]:
model.train() # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

In [8]:
# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj" ,"ffn"],
    lora_dropout=0.2,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()

trainable params: 2,097,152 || all params: 264,507,392 || trainable%: 0.7929


Prempare Dataset

In [9]:
from datasets import load_dataset

dataset = load_dataset('s200862/medical_qa_meds', split='train')

In [10]:
import pandas as pd
from datasets import Dataset

# Assuming 'data' is your dataset
# Convert to Pandas DataFrame for easy manipulation
data_df = pd.DataFrame(dataset)

# Splitting the data
train_df = data_df.iloc[:1000]  # First 1000 rows for training
validation_df = data_df.iloc[1000:2000]  # Next 1000 rows for validation
test_df = data_df.iloc[2000:3000]  # Remaining 1753 rows for testing

# Convert back to Dataset
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)
test_dataset = Dataset.from_pandas(test_df)

# Output the datasets
train_dataset, validation_dataset, test_dataset

(Dataset({
     features: ['text'],
     num_rows: 1000
 }),
 Dataset({
     features: ['text'],
     num_rows: 1000
 }),
 Dataset({
     features: ['text'],
     num_rows: 1000
 }))

In [11]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["text"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

# tokenize training and validation datasets
tokenized_train_data = train_dataset.map(tokenize_function, batched=True)
tokenized__val_data = validation_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [12]:
# setting pad token
tokenizer.pad_token = tokenizer.eos_token

# data collator
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer,
                                                              mlm=False)

In [13]:
# hyperparameters
lr = 1e-4
batch_size = 10
num_epochs = 10

# define training arguments
training_args = transformers.TrainingArguments(
    output_dir= "medicalGPT",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",
)

/DATA/sriparna/TA/Daivik/.conda/lib/python3.9/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
# configure trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized__val_data,
    args=training_args,
    data_collator=data_collator
)

# train model
model.config.use_cache = False  # silence the warnings.

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [15]:
trainer.train()

/DATA/sriparna/TA/Daivik/.conda/lib/python3.9/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,2.039400,1.655010
2,1.443800,1.640315
4,1.279800,1.676756
6,1.163700,1.745356
8,1.072800,1.796952
9,1.024600,1.811427


/DATA/sriparna/TA/Daivik/.conda/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/DATA/sriparna/TA/Daivik/.conda/lib/python3.9/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/DATA/sriparna/TA/Daivik/.conda/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when poss

TrainOutput(global_step=620, training_loss=1.3176573599538495, metrics={'train_runtime': 4901.4272, 'train_samples_per_second': 2.04, 'train_steps_per_second': 0.126, 'total_flos': 2171251091275776.0, 'train_loss': 1.3176573599538495, 'epoch': 9.92})

In [16]:
# renable warnings
model.config.use_cache = True

In [17]:
new_model = "mistral-7B-Inst-finetune"

In [18]:
trainer.model.save_pretrained(new_model)

/DATA/sriparna/TA/Daivik/.conda/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
# # load model from hub
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

config = PeftConfig.from_pretrained("mistral-7B-Inst-finetune")
model = PeftModel.from_pretrained(model, "mistral-7B-Inst-finetune")

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

/DATA/sriparna/TA/Daivik/.conda/lib/python3.9/site-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_p

In [21]:
# Empty VRAM
del model
# del pipe
del trainer
import gc
gc.collect()
gc.collect()

0

In [25]:
# model = model.merge_and_unload()

In [27]:
print(1)

1


In [29]:


model.push_to_hub("mistral-7B-Inst-finetune", check_pr=True)

tokenizer.push_to_hub("mistral-7B-Inst-finetune",check_pr=True)

/DATA/sriparna/TA/Daivik/.conda/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Prajapat/mistral-7B-Inst-finetune/commit/538478a8d163dccc0d1532b5cb7bcac114b31a4f', commit_message='Upload tokenizer', commit_description='', oid='538478a8d163dccc0d1532b5cb7bcac114b31a4f', pr_url=None, pr_revision=None, pr_num=None)

: 